## Spotify App Reviews Sentiment Analysis

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [22]:
stf = pd.read_csv('reviews.csv')

### Data PreProcessing

In [4]:
!pip install nltk gensim

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 160.7 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 75.2 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 KB 160.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 84.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01


In [23]:
import gensim
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces, preprocess_string, split_alphanum, strip_short, strip_numeric
import re 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/canada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def lower_case(docs):
  return [doc.lower() for doc in docs]

# Loai bo dau cau & ki tu dac biet
def remove_punc(docs):
  return [strip_non_alphanum(doc).strip() for doc in docs]

# Tach cac so va chu lien nhau
def separate_num(docs):
  return [split_alphanum(doc) for doc in docs]

def remove_one_letter_word(docs):
  return [strip_short(doc) for doc in docs]

def remove_number(docs):
  return [strip_numeric(doc) for doc in docs]

# Thay the nhieu spaces bang mot space
def replace_multiple_whitespaces(docs):
  return [strip_multiple_whitespaces(doc) for doc in docs]

def remove_stopwords(docs):
    return [" ".join([word for word in doc.split() if word not in stop_words]) for doc in docs]

doc = lower_case(stf['Review'])
doc = remove_punc(doc)
doc = separate_num(doc)
doc = remove_one_letter_word(doc)
doc = remove_number(doc)
doc = replace_multiple_whitespaces(doc)
doc = remove_stopwords(doc)

stf['Review'] = doc

In [25]:
stf.head()

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,great music service audio high quality app eas...,5,2,NaN
1,2022-07-09 14:21:22,please ignore previous negative rating app sup...,5,1,NaN
2,2022-07-09 13:27:32,pop get best spotify experience android annoyi...,4,0,NaN
3,2022-07-09 13:26:45,really buggy terrible use recently,1,1,NaN
4,2022-07-09 13:20:49,dear spotify get songs put playlist shuffle play,1,1,NaN


In [26]:
stf1 = stf.copy()
rate = [1,2,3,4,5]
sentiment = [0, 0, 1, 1, 1]
for i in range(5):
    stf1["Rating"].replace(rate[i], sentiment[i],inplace=True)

In [35]:
stf1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61594 entries, 0 to 61593
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Time_submitted  61594 non-null  object
 1   Review          61594 non-null  object
 2   Rating          61594 non-null  int64 
 3   Total_thumbsup  61594 non-null  int64 
 4   Reply           216 non-null    object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


### SVM

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the data
# df = pd.read_csv("review.csv")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(stf1['Review'], stf1['Rating'], test_size=0.2, random_state=42)

# Transform the text data into numerical features using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

# Train an SVM classifier on the training data
clf = SVC(kernel='linear', verbose=True)
clf.fit(X_train_vectors, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_vectors)

# Evaluate the model's accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

[LibSVM]................................*..................*.*
optimization finished, #iter = 51535
obj = -17712.833044, rho = 0.451040
nSV = 21613, nBSV = 17734
Total nSV = 21613
Accuracy: 0.8262846010228103


In [18]:
# Load new data
new_data = ["terrible"]

# Transform the new data into numerical features using the same TfidfVectorizer
new_data_vectors = vectorizer.transform(new_data)

# Make predictions on the new data
new_predictions = clf.predict(new_data_vectors)

# Display the predictions
print("Predictions:", new_predictions)

Predictions: ['negative']


In [24]:
from sklearn.preprocessing import LabelEncoder

# Encode the labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Make predictions on the test data
y_pred = clf.predict(X_test_vectors)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='binary')
print("F1 score:", f1)

ValueError: invalid literal for int() with base 10: 'negative'

### Logistic Regression

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(stf1['Review'], stf1['Rating'], test_size=0.2, random_state=42)

# Transform the text data into numerical features using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

# Train a Logistic Regression classifier on the training data
clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train_vectors, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_vectors)

# Evaluate the model's accuracy
train_accuracy = accuracy_score(y_train, clf.predict(X_train_vectors))
test_accuracy = accuracy_score(y_test, y_pred)

print("Train accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)

Train accuracy: 0.8558295281582953
Test accuracy: 0.8318045295884406


In [36]:
# Load new data
new_data = ["not"]

# Transform the new data into numerical features using the same TfidfVectorizer
new_data_vectors = vectorizer.transform(new_data)

# Make predictions on the new data
new_predictions = clf.predict(new_data_vectors)

# Display the predictions
print("Predictions:", new_predictions)

Predictions: ['positive']


### Random Forest & Naive Bayes

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [29]:
X_train, X_test, y_train, y_test = train_test_split(stf1['Review'], stf1['Rating'], test_size=0.2, random_state=42)

In [30]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [31]:
rf2 = RandomForestClassifier()
rf2.fit(X_train_tfidf, y_train)
rf_tfidf = rf2.score(X_test_tfidf, y_test)
print(f'Random Forest Classifier on TF-IDF Vectors: {rf_tfidf}')

Random Forest Classifier on TF-IDF Vectors: 0.8128094812890657


In [32]:
mnb2 = MultinomialNB()
mnb2.fit(X_train_tfidf, y_train)
mnb_tfidf = mnb2.score(X_test_tfidf, y_test)
print(f'Multinomial Naive Bayes Classifier on TF-IDF Vectors: {mnb_tfidf}')

Multinomial Naive Bayes Classifier on TF-IDF Vectors: 0.8098871661660849


In [33]:
model = {'Model':['RandomForestClassifier-TFIDFVectors', 
                  'MultinomialNBClassifier-TFIDFVectors'],
         'Score':[rf_tfidf,mnb_tfidf]
         }
model_df = pd.DataFrame(model)
model_df

,Model,Score
0,RandomForestClassifier-TFIDFVectors,0.812809
1,MultinomialNBClassifier-TFIDFVectors,0.809887
